#### Jupyter Notebook to scrape kml files from the Forest clearance portal, Ministry of Environment and Forests, India.  
##### This notebook downloads all the files in a single folder. 

In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import pathlib
import urllib.parse

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [104]:
def downloadKMLForProject(project_id):
    
    response = requests.get('http://forestsclearance.nic.in/viewreport.aspx?pid=' + project_id)
    soup = BeautifulSoup(response.text, "html.parser")
    
    folder = os. getcwd() + "/KMLS_NEW/" + project_id.replace('/','_') 
    
    if(os.path.isdir(folder) == False):
        os.mkdir(folder)
        
    # Get all <a> tags containing href. 
    links = soup.find_all('a', href=True)
    
    for link in links:
        
        # Extract href from <a>
        href = link['href']
        
        # Parse href to get query string. 
        url = urllib.parse.urlparse(href)
        
        # Parse query string to get file name only. 
        params = urllib.parse.parse_qs(url.query)
         
        if 'FileName' in params: 
            filename = project_id.replace('/','_') + '_' + params['FileName'][0] 
        
        # Download KML files only.
        if (pathlib.Path(filename).suffix == '.kml'):        
            downloadURL = requests.get('http://forestsclearance.nic.in/'+ href)
            file = folder + '/' + filename    
            open(file, 'wb').write(downloadURL.content)
            
    return

In [105]:
df = pd.read_excel(os. getcwd() + '/Records.xlsx', index_col=0)

for index, row in df.iterrows():
    downloadKMLForProject(row['Proposal_no'])

KeyboardInterrupt: 